# データベースチューニングを語る

## 前置き

- 本トークで出てくる言葉は、あまり信用しないでください。
- それっぽい言葉が、出てきますが、ほとんど私の経験則に基づく言葉です。
- 言葉自体は、私の言葉なので、意味は、信用しないでください。
- 書籍などから正しい言葉などを引用してないので、一般的な言葉では無い可能性ががあります。
- 一応、信用していい部分、悪い分をトークの途中で入れていきながら進めたいと思っています。


## お話すること

### 主な内容
- データベースのチューニング方法とその効果に関して、
- 私の経験則に基づき、整理・検討してみた結果を、
- お話してみたいと思います。

### 話す順番
1. データベースのチューニング方法の羅列
2. 評価軸の整理
3. コスト分類ごとの特色の検討
4. 方法と評価軸の整理(まとめ)

## 1. データベースチューニング方法の羅列


- プログラム的な対応(カーソルループの廃止など)
- 等価なSQLへの書き換え
- ヒント文の追加
- インデックス
- ビューの実体化(オラクルで言うマテビュー)
- パーティショニング
- リードレプリカ
- パラメータ変更(キャッシュサイズ調整、ファイルの分散配置、パラレル度調整、etc.)
- ハードウェア増強(メモリ、CPU、I/O)



** (順番にはあまり意味はない。なんとなく上のほうがアプリ寄り、下のほうがインフラ寄り) **



## 2.評価軸の整理

- 私は、(改めて考えてみるに)、
- 基本的には、(直感的な)**<u>コスト対効果</u>**で、
- 最適なチューニング方法を評価・選定している(してきた)ようです。



### 効果とは?

- 処理速度が向上すること、および、その向上度合い(どの程度向上するか)

### コストとは?

まず以下の３点に大別されるように思う。

1. チューナーのコスト
  - 主に、チューナーが調査・検証・効果測定する工数
  - 要するに私が動くコスト
2. **チューナー以外のコスト => 後述　**
3. 管理コスト(主題から逸れるので無視する)



## チューナー以外のコストとは?(コストの分類)

- チューナーが、チューニング案を出したあと、本番の対応にかかるコストを指します。
- たぶん、さらに以下の３つに分けられるように思います。


1. アプリ変更コスト(おおむね、アプリエンジニアの工数のように思われる)
  - 設計・実装・テスト作業(単体→結合→システム)
  - 本番リリース作業
  - リリース後の立会、結果報告
  - etc.
2. インフラ増強コスト
  - そもそも増やせるのかの検討確認作業
  - 増やすための設計・実装・テスト作業
  - モノの料金(メモリ、CPU、I/O)
  - コストだけでなく調達のためのリードタイムが必要(クラウドならゼロにできる)
3. **ミドル(DBMS)変更コスト(パラメータ変更やパーティショニングなど)**　＝＞後述


## チューナー以外のコストとは?(コストの分類：３つめ)

### 3.**ミドル(DBMS)変更コスト(パラメータ変更やパーティショニングなど)**
  - アプリ変更、インフラ増強の両方を伴いかねない(伴う場合が多い)
  - たとえば
    - パーティショニングするなら、パーティションキーの検討・設計が最低限必要。
      - これは「アプリ視点」でないと、できない(ので、アプリ技術者が動く必要あり) 
      - 場合によってはパーティションをスライドさせるためのアプリが必要
      - 物理的な分散配置のためのストレージ設計をインフラ技術者と詰めなければいけない



## 3.コスト分類ごとの特色

#### 1.アプリ変更の特色
  - ○アプリ側だけのコスト負担や都合でなんとかなるような気がする。
  - ○その意味では、打つ手としては、比較的、打ちやすい。
  - ×一方で、コストやリードタイムが読みにくい、というリスクがある。
  - ×また対応できる時間が限られている可能性がある
    - プログラムをいじれるパートナーさんの契約期間とか.

#### 2.インフラ増強の特色
  - ○インフラ側だけのコスト負担や都合でなんとかなるような気がする。
  - ○その意味では、打つ手としては、まあ一応、打ちやすい。
  - ○そもそも、ちゃんとインフラ容量設計をやっていれば、安全率を十分にかけているので、増強の必要性が発生しにくい。
  - ×一方で、リアルな料金が発生する。（増強の費用はたいてい顧客持ちなので、説明できないと行けない)
  - ×調達のためのリードタイムが発生する。
    - クラウドならゼロにできるが、長い目で見れば料金は高額になる
  


## コスト分類ごとの特色 3つめ

#### 3.ミドル(DBMS)変更コスト
  - アプリ変更、インフラ増強の両方を伴いかねない(伴う場合が多い)
  - だから、できればチューニングでミドル変更はやりたくない
  - **最初にきちんと評価・設計して決めておくことが大事**
  - 最近のDBMSは「デフォルト設定」で大体なんとかなるくらいに賢い
    - が、**思わぬ落とし穴には気をつけたい**
      - 技術は進化?退化?変化?する
      - DBA(ないしDA)の役割を持つ人(DBスペシャリスト)が、本当は、すべての開発プロジェクトにおいて必要なのではないだろうか。


## 4.方法と評価軸の整理(まとめ)


|チューニング方法|想定効果|コスト分類|動く人|想定コスト量|コスト対効果|
|:----:|:----:|:----:|:----:|:----:|:----:|
|プログラム的な対応|大|アプリ変更|アプリ|大|△|
|等価なSQLへの書き換え|大|アプリ変更|アプリ+チューナー|中|○|
|ヒント文の追加|中or大|チューナーのコスト|アプリ+チューナー|小|○|
|インデックス|中or大|チューナーのコスト|チューナー|小|◎|
|ビューの実体化|不明 うまく行けば大|ミドル変更|全|不明 大の方が無難|評価できず|
|パーティショニング|不明 うまく行けば大|ミドル変更|全|不明 大の方が無難|評価できず|
|リードレプリカ|不明 うまく行けば大|ミドル変更|全|不明 大の方が無難|評価できず|
|パラメータ変更|不明 うまく行けば大|ミドル変更|インフラ+チューナー|中|△|
|ハードウェア増強|小よくて中|ハードウェア増強|インフラ|中or大|△|


全=アプリ、インフラ、ミドル、チューナー

### 結論

#### ミドル以下はきちんと設計しておきましょう。
#### 特にミドル変更は、後でチューニングで、なんとかなるものでは、無い(と、考えといたほうが、無難)。


## パワポに戻る